# [Scene Recognition with Deep Learning](https://dellaert.github.io/19F-4476/proj6.html)
Remember in the previous project, where you have tried with a bunch of old-school techniques for image classification on the 15-scene data? Well for this project we are going to focus on the same task, but with the state-of-the-art approach: deep learning.

Basic learning objectives of this project:
1. Construct the fundamental pipeline for performing deep learning using PyTorch;
2. Understand the concepts behind different layers, optimizers, and learning schedules;
3. Experiment with different models and observe the performance.

The starter code is mostly initialized to 'placeholder' just so that the starter
code does not crash when run unmodified and you can get a preview of how
results are presented.

In [1]:
from proj6_code.runner import Trainer
from proj6_code.optimizer import get_optimizer
from proj6_code.simple_net import SimpleNet
from proj6_code.simple_net_dropout import SimpleNetDropout
from proj6_code.my_alexnet import MyAlexNet
from proj6_code.image_loader import ImageLoader
from proj6_code.data_transforms import get_fundamental_transforms, get_data_augmentation_transforms
from proj6_code.stats_helper import compute_mean_and_std

import torch

%load_ext autoreload
%autoreload 2

In [2]:
from proj6_unit_tests.test_base import verify
from proj6_unit_tests.test_stats_helper import test_mean_and_variance
from proj6_unit_tests.test_image_loader import test_dataset_length, test_unique_vals, test_class_values, test_load_img_from_path
from proj6_unit_tests.test_data_transforms import test_fundamental_transforms
from proj6_unit_tests.test_dl_utils import test_predict_labels, test_compute_loss
from proj6_unit_tests.test_simple_net import test_simple_net
from proj6_unit_tests.test_simple_net_dropout import test_simple_net_dropout
from proj6_unit_tests.test_my_alexnet import test_my_alexnet

In [3]:
is_cuda = True
is_cuda = is_cuda and torch.cuda.is_available() # will turn off cuda if the machine doesnt have a GPU

## Part 1: Train a SimpleNet
To train a network in PyTorch, we need 4 components:
1. **Dataset** - an object which can load the data and labels given an index.
2. **Model** - an object that contains the network architecture definition.
3. **Loss function** - a function that measures how far the network output is from the ground truth label.
4. **Optimizer** - an object that optimizes the network parameters to reduce the loss value.

### Part 1.1: Datasets
One crucial aspect of deep learning is to perform data preprocessing. In this project we are going to "zero-center" and "normalize" the dataset. 

### Compute mean and standard deviation of the dataset
To begin with, fill in the `compute_mean_and_std()` in `stats_helper.py` to compute the **mean** and **standard deviation** of both training and validation data.

In [ ]:
print("Testing your mean and std computation: ", verify(test_mean_and_variance))
dataset_mean, dataset_std = compute_mean_and_std('../data/')

In [ ]:
print('Dataset mean = {}, standard deviation = {}'.format(dataset_mean, dataset_std))

Now let's create the **Datasets** object to be used later. Remember back in Project 1, we have initialized such a class to load 5 images? Here the task is similar: download the data into the project folder, and then complete the `image_loader.py`. The essence is to retrieve the paths to all the images required, and be able to provide the **path** and the **class id** when given an index.

In [ ]:
inp_size = (64,64)
print("Testing your image loader (length):", verify(test_dataset_length))
print("Testing your image loader (values):", verify(test_unique_vals))
print("Testing your image loader (classes):", verify(test_class_values))
print("Testing your image loader (paths):", verify(test_load_img_from_path))

### Data transforms
For this part, complete the function `get_fundamental_transforms()` in `data_transforms.py` to compile a list of fundamental transforms which:
1. Resize the input image to the desired shape;
2. Convert it to a tensor;
3. Normalize them based on the computed mean and standard deviation.

In [ ]:
print("Testing your fundamental data transforms: ", verify(test_fundamental_transforms))

### Part 1.2: Model
The data is ready! Now we are preparing to move to the actual core of deep learning: the architecture. To get you started in this part, simply define a **2-layer** model in the `simple_net.py`. Here by "2 layers" we mean **2 convolutional layers**, so you need to figure out the supporting utilities like ReLU, Max Pooling, and Fully Connected layers, and configure them with proper parameters to make the tensor flow.

You may refer to the project [instruction page](https://dellaert.github.io/19F-4476/proj6.html) for a sample network architecture (it's the architecture TAs used in their implementation and is sufficient to get you pass Part 1).

In [ ]:
print("Testing your SimpleNet architecture: ", verify(test_simple_net))

In [ ]:
simple_model = SimpleNet()

### Loss function
When defining your model architecture, also initialize the `loss_criterion` variable there. Remeber this is multi-class classification problem, and think about what loss function might be useful here.

In [ ]:
print(simple_model)

### Optimizer
Next, **initialize the following cell with proper values for learning rate and weight decay** (you can come back and tune these values for better performance once the trainer section is done), and then fill in the `optimizer.py` to initialize a basic optimization function; this should only take a few lines of code.

In [ ]:
# TODO: add a decent initial setting and tune from there
optimizer_config = {
  "optimizer_type": "sgd",
  "lr": 1e-10,
  "weight_decay": 1e-1
}

In [ ]:
optimizer = get_optimizer(simple_model, optimizer_config)

### Part 1.3: Trainer
Next we define the trainer for the model; to start, complete the `predict_labels()` and `compute_loss()` in `dl_utils.py`: given a model, compute the corresponding predictions and loss respectively.

In [ ]:
print("Testing your trainer (model prediction): ", verify(test_predict_labels))
print("Testing your trainer (loss values): ", verify(test_compute_loss))

Then pass in the model, optimizer, transforms for both the training and testing datasets into the trainer, and proceed to the next cell to train it. If you have implemented everything correctly, you should be seeing a decreasing loss value.

**Note** that your CPU should be sufficient to handle the training process for all networks in this project, and the following training cells will take less than 5 minutes; you may also want to decrease the value for `num_epochs` and quickly experiment with your parameters. The default value of **30** is good enough to get you around the threshold for Part 1, and you are free to increase it a bit and adjust other parameters in this part.

In [ ]:
trainer = Trainer(data_dir='../data/', 
                  model = simple_model,
                  optimizer = optimizer,
                  model_dir = 'model_checkpoints/simple_net',
                  train_data_transforms = get_fundamental_transforms(inp_size, dataset_mean, dataset_std),
                  test_data_transforms = get_fundamental_transforms(inp_size, dataset_mean, dataset_std),
                  batch_size = 32,
                  load_from_disk = False,
                  cuda = is_cuda
                 )

In [ ]:
trainer.train(num_epochs=30)

After you have finished the training process, now plot out the loss and accuracy history. You can also check out the final accuracy for both training and testing data. Copy the accuracy plots and values onto the report, and answer the questions there. Note that you are required to obtain a **50%** testing accuracy to receive full credits for Part 1.

In [ ]:
trainer.plot_loss_history()
trainer.plot_accuracy()

In [ ]:
train_accuracy = trainer.train_accuracy_history[-1]
validation_accuracy = trainer.validation_accuracy_history[-1]
print('Train Accuracy = {}; Validation Accuracy = {}'.format(train_accuracy, validation_accuracy))

## Part 2: Overfitting
Feeling good? We have easily obtained a 50% accuracy on the testing data with a very simple model; this is already the highest standard for the previous project. Feeling even better for the training accuracy right? More than 90% (if you have implemented everything correctly). But should you?

Our final accuracies for training and testing data differ a lot from each other, which indicates that the model we have defined **fits too well with the training data, but is unable to generalize well on data it has never seen before**: this is often regarded as **overfitting**. In this section we are going to apply 2 techniques to tackle with it: adjusting both data and model.

### Part 2.1: Jitter, Random Flip, and Normalization
One common technique to increase the "variability" of the data is to **augment** it. Firstly, we don't have a huge amount of data, so let's "jitter" based on it; secondly, when you mirror an image of a **kitchen**, you can tell that the mirrored image is still a kitchen. Hence, finsih the `get_data_augmentation_transforms()` function in `data_transforms.py`: you may first copy your existing fundamental transform implementation into this function, and then insert a couple of other transforms which help you do the above adjustment.

In [ ]:
inp_size = (64,64)

### Part 2.2: Dropout
"Dropout" is a technique commonly used to regularize the network. It randomly turns off the connection between neurons inside the network and prevent the network from relying too much on a specific neuron. Follow the instruction and finish the `simple_net_dropout.py` with your previous SimpleNet model, plus the dropout layer, and lastly re-run the training process as below.

In [ ]:
print("Testing your SimpleNetDropout architecture: ", verify(test_simple_net_dropout))

In [ ]:
simple_model_dropout = SimpleNetDropout()
print(simple_model_dropout)

Similar to the previous part, **initialize the following cell with proper values for learning rate and weight decay**.

In [ ]:
# TODO: add a decent initial setting and tune from there
optimizer_config = {
  "optimizer_type": "sgd",
  "lr": 1e-10,
  "weight_decay": 1e-1
}

In [ ]:
optimizer = get_optimizer(simple_model_dropout, optimizer_config)

In [ ]:
trainer = Trainer(data_dir='../data/', 
                  model = simple_model_dropout,
                  optimizer = optimizer,
                  model_dir = 'model_checkpoints/simple_net_dropout',
                  train_data_transforms = get_data_augmentation_transforms(inp_size, dataset_mean, dataset_std),
                  test_data_transforms = get_fundamental_transforms(inp_size, dataset_mean, dataset_std),
                  batch_size = 32,
                  load_from_disk = False,
                  cuda = is_cuda
                 )

The following cell will take longer than Part 1, as now we have more data (and more variability), and the model is slightly more complicated than before as well; however, it should finish within 10~15 minutes anyway, and the default `num_epochs` is also good enough as a starting point for you to pass this part.

In [ ]:
trainer.train(num_epochs=30)

Similar to the previous part, now plot out the loss and accuracy history. You'll need to pass a threshold of **55%** to receive full credits for this part. Also copy the plots onto the report, and answer the questions accordingly.

In [ ]:
trainer.plot_loss_history()
trainer.plot_accuracy()

In [ ]:
train_accuracy = trainer.train_accuracy_history[-1]
validation_accuracy = trainer.validation_accuracy_history[-1]
print('Train Accuracy = {}; Validation Accuracy = {}'.format(train_accuracy, validation_accuracy))

## Part 3: AlexNet
You can see that after the above adjustment, our model performance increases in terms of testing accuracy. Although the training accuracy drops, now it's closer to the testing values and that's more natural in terms of performance. But we are not satisfied with the final performance yet. Our model, in the end, is still a 2-layer SimpleNet and it might be capable of capturing some features, but could be improved a lot if we go **deeper**. In this part we are going to see the power of a famous model: AlexNet.

In [ ]:
inp_size = (224, 224)

### Part 3.1 & 3.2: Fine-tuning the AlexNet
Now switch to `my_alexnet.py`, and define a AlexNet which can be fit onto our dataset: PyTorch has provided us with pre-trained models like AlexNet, so what you want to do is to load the model first, and then adjust some of the layers such that it fits with our own dataset, instead of outputing scores to 1000 classes from the original AlexNet model.

After you have defined the correct architecture of the model, make some tweaks to the existing layers: **freeze** the **convolutional** layers and first 2 **linear** layers so we don't update the weights of them; more details can be found in the instruction webpage.

In [ ]:
print("Testing your AlexNet architecture: ", verify(test_my_alexnet))

In [ ]:
my_alexnet = MyAlexNet()
print(my_alexnet)

In [ ]:
# TODO: add a decent initial setting and tune from there
optimizer_config = {
  "optimizer_type": "sgd",
  "lr": 1e-10,
  "weight_decay": 1e-1
}

In [ ]:
optimizer = get_optimizer(my_alexnet, optimizer_config)

In [ ]:
trainer = Trainer(data_dir='../data/', 
                  model = my_alexnet,
                  optimizer = optimizer,
                  model_dir = 'model_checkpoints/myalexnet/',
                  train_data_transforms = get_data_augmentation_transforms(inp_size, dataset_mean, dataset_std),
                  test_data_transforms = get_fundamental_transforms(inp_size, dataset_mean, dataset_std),
                  batch_size = 32,
                  load_from_disk = False,
                  cuda = is_cuda
                 )

The following training cell will take roughly 20 minutes or slightly more using CPU (but possibly under 5 minute using GPU depending on the batch size; the TAs got it within 3 minutes on a GTX1060).

In [ ]:
trainer.train(num_epochs=5)

Like both previous sections, you are required to pass a threshold of **85%** for this part. Copy the plots and values onto the report and answer questions accordingly.

In [ ]:
trainer.plot_loss_history()
trainer.plot_accuracy()

In [ ]:
train_accuracy = trainer.train_accuracy_history[-1]
validation_accuracy = trainer.validation_accuracy_history[-1]
print('Train Accuracy = {}; Validation Accuracy = {}'.format(train_accuracy, validation_accuracy))

With all these we have concluded the last project of CS4476 Computer Vision. Things might be hard along the way, but we hope you enjoyed this journey and have learnt something in this field. Our team has learnt a lot from you guys as well, so thank you and wish you all the best in your future endeavors!